In [1]:
import openpyxl

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os


In [3]:
def process_dataframe(input_file):
    # Read the Excel file with the correct header row (row 35)
    df = pd.read_excel(
        r"C:\Users\Parth1\.ipython\Trading journal\Trades_History_P59962265 1-10].xlsx",
        header=34  # Python uses 0-based indexing, so row 35 is index 34
    )
    
    # Display basic information about the dataframe
    print("DataFrame Shape:", df.shape)
    print("\nColumn Names:")
    for i, col in enumerate(df.columns, 1):
        print(f"{i}. {col}")
    
    print("\nData Types:")
    print(df.dtypes)
    
    print("\nFirst few rows of actual data:")
    print(df.head())
    
    return df

# Call the function to analyze the Excel file
df = process_dataframe("Trades_History_P59962265 1-10].xlsx")

DataFrame Shape: (20, 19)

Column Names:
1. Scrip/Contract
2. Buy/Sell
3. Buy Price
4. Sell Price
5. Quantity
6. Brokerage
7. GST
8. STT
9. Sebi Tax
10. Exchange Turnover Charges
11. Stamp Duty
12. Other Charges
13. IPFT Charges
14. Order Type
15. Segment
16. Exchange
17. Order ID
18. Trade ID
19. Date

Data Types:
Scrip/Contract                       object
Buy/Sell                             object
Buy Price                           float64
Sell Price                          float64
Quantity                              int64
Brokerage                             int64
GST                                 float64
STT                                 float64
Sebi Tax                            float64
Exchange Turnover Charges           float64
Stamp Duty                          float64
Other Charges                         int64
IPFT Charges                        float64
Order Type                           object
Segment                              object
Exchange               

C:\Users\Parth1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# Run the analysis to see the columns and data structure
df = process_dataframe("Trades_History_P59962265 1-10].xlsx")


DataFrame Shape: (20, 19)

Column Names:
1. Scrip/Contract
2. Buy/Sell
3. Buy Price
4. Sell Price
5. Quantity
6. Brokerage
7. GST
8. STT
9. Sebi Tax
10. Exchange Turnover Charges
11. Stamp Duty
12. Other Charges
13. IPFT Charges
14. Order Type
15. Segment
16. Exchange
17. Order ID
18. Trade ID
19. Date

Data Types:
Scrip/Contract                       object
Buy/Sell                             object
Buy Price                           float64
Sell Price                          float64
Quantity                              int64
Brokerage                             int64
GST                                 float64
STT                                 float64
Sebi Tax                            float64
Exchange Turnover Charges           float64
Stamp Duty                          float64
Other Charges                         int64
IPFT Charges                        float64
Order Type                           object
Segment                              object
Exchange               

C:\Users\Parth1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
def process_trading_data(df):
    # Group by Scrip/Contract to match buy and sell transactions
    grouped = df.groupby('Scrip/Contract')
    
    results = []
    for name, group in grouped:
        buy_data = group[group['Buy Price'].notna()]
        sell_data = group[group['Sell Price'].notna()]
        
        # Calculate basic amounts
        buy_amount = (buy_data['Buy Price'] * buy_data['Quantity']).sum()
        sell_amount = (sell_data['Sell Price'] * sell_data['Quantity']).sum()
        
        # Sum all charges for the group
        total_charges = (
            group['Brokerage'].sum() +
            group['GST'].sum() +
            group['STT'].sum() +
            group['Sebi Tax'].sum() +
            group['Exchange Turnover Charges'].sum() +
            group['Stamp Duty'].sum() +
            group['Other Charges'].sum() +
            group['IPFT Charges'].sum()
        )
        
        # Calculate net P&L after all charges
        net_pnl = sell_amount - buy_amount - total_charges
        
        # Only include if there are both buy and sell transactions
        if buy_amount > 0:
            profit_percentage = (net_pnl / buy_amount * 100) if buy_amount != 0 else 0
            
            results.append({
                'Scrip/Contract': name,
                'Buy_Amount': buy_amount,
                'Buy Price': buy_data['Buy Price'].mean(),
                'Sell Price': sell_data['Sell Price'].mean(),
                'Total_Charges': total_charges,
                'Net_PnL': net_pnl,
                'Profit_Percentage': profit_percentage
            })
    
    # Create final dataframe
    pnl_metrics = pd.DataFrame(results)
    
    # Round numerical columns
    numeric_columns = ['Buy_Amount','Buy Price', 'Sell Price','Total_Charges', 'Net_PnL', 'Profit_Percentage']
    pnl_metrics[numeric_columns] = pnl_metrics[numeric_columns].round(2)
    
    # Display results
    print("\nPnL Metrics Table:")
    print("=" * 80)
    display(pnl_metrics)
    
    # Summary statistics
    print("\nSummary:")
    print(f"Total Investment: ₹{pnl_metrics['Buy_Amount'].sum():,.2f}")
    print(f"Total Charges: ₹{pnl_metrics['Total_Charges'].sum():,.2f}")
    print(f"Total Net P&L: ₹{pnl_metrics['Net_PnL'].sum():,.2f}")
    print(f"Overall Return: {(pnl_metrics['Net_PnL'].sum() / pnl_metrics['Buy_Amount'].sum() * 100):,.2f}%")
    
    return pnl_metrics

# Process the data
pnl_metrics = process_trading_data(df)


PnL Metrics Table:


,Scrip/Contract,Buy_Amount,Buy Price,Sell Price,Total_Charges,Net_PnL,Profit_Percentage
0,BHARAT ELECTRONICS LTD,183375.00,407.50,NaN,241.46,-183616.46,-100.13
1,GODREJ PROPERTIES LTD,162065.90,2025.88,2035.00,408.34,325.76,0.20
2,OPTIDX NIFTY Oct 7 2025 24800.00 CE (BT),10226.25,68.18,70.47,66.51,278.49,2.72
3,OPTIDX NIFTY Oct 7 2025 25200.00 CE (BT),708.75,4.72,3.55,48.15,-224.40,-31.66
4,UTI ASSET MNGMT CO LTD,184450.00,1317.50,1327.00,136.67,1193.33,0.65



Summary:
Total Investment: ₹540,825.90
Total Charges: ₹901.13
Total Net P&L: ₹-182,043.28
Overall Return: -33.66%
